# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Import files
weather_df_path = "../output_data/weather_df.csv"

# Read the mouse data and the study results
weather_df = pd.read_csv(weather_df_path)
weather_df


,City,Latitude,Longitude,Max Temp_F,Humidity_%,Cloudiness_%,Wind Speed,Country,Date
0,Magadan,59.5667,150.8000,-4.79,85,70,14.63,RU,12-13-2020
1,Los Llanos de Aridane,28.6585,-17.9182,63.59,84,56,6.51,ES,12-13-2020
2,Chui,-33.6971,-53.4616,62.60,73,75,5.91,UY,12-13-2020
3,Puerto Ayora,-0.7393,-90.3518,72.77,89,99,8.61,EC,12-13-2020
4,Volkhov,59.9258,32.3382,23.72,98,98,3.42,RU,12-13-2020
...,...,...,...,...,...,...,...,...,...
592,Aginskoye,51.1000,114.5300,-5.73,82,0,6.62,RU,12-13-2020
593,Santa Cruz de Tenerife,28.4682,-16.2546,65.14,84,53,4.59,ES,12-13-2020
594,Key Largo,25.0865,-80.4473,75.11,73,10,9.31,US,12-13-2020
595,Barsovo,61.1667,73.1667,14.36,95,69,10.40,RU,12-13-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)
locations
#weather_df = weather_df.dropna()

,Latitude,Longitude
0,59.5667,150.8000
1,28.6585,-17.9182
2,-33.6971,-53.4616
3,-0.7393,-90.3518
4,59.9258,32.3382
...,...,...
592,51.1000,114.5300
593,28.4682,-16.2546
594,25.0865,-80.4473
595,61.1667,73.1667


In [14]:
# Humidity rate - drop NaN
humidity_rate = weather_df["Humidity_%"].astype(float)
humidity_rate

0      85.0
1      84.0
2      73.0
3      89.0
4      98.0
       ... 
592    82.0
593    84.0
594    73.0
595    95.0
596    59.0
Name: Humidity_%, Length: 597, dtype: float64

In [37]:
weather_df.dtypes

City             object
Latitude        float64
Longitude       float64
Max Temp_F      float64
Humidity_%        int64
Cloudiness_%      int64
Wind Speed      float64
Country          object
Date             object
dtype: object

In [35]:
# Plot Heatmap
#https://jupyter-gmaps.readthedocs.io/en/latest/api.html
fig = gmaps.figure(map_type="HYBRID",center=(45.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# # save the plot
# plt.savefig("../output_data/map")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70 degrees.
temperature_one = weather_df.loc[weather_df["Max Temp_F"]<80,:]
temperature_two = temperature_one.loc[weather_df["Max Temp_F"]>70,:]
temperature_two.head()

#Wind speed less than 10 mph.
wind_speed_tree = temperature_two.loc[temperature_two["Wind Speed"]<10,:]
wind_speed_tree

#Zero cloudiness.
idea_weather_df = wind_speed_tree.loc[wind_speed_tree["Cloudiness_%"]==0,:]
idea_weather_df

,City,Latitude,Longitude,Max Temp_F,Humidity_%,Cloudiness_%,Wind Speed,Country,Date
210,Mandera,3.9366,41.8670,75.34,28,0,3.49,KE,12-13-2020
251,Betioky,-23.7167,44.3833,74.25,63,0,7.45,MG,12-13-2020
252,Kodār,16.9833,79.9667,72.19,81,0,2.68,IN,12-13-2020
464,Jizan,17.3333,42.6667,76.71,74,0,3.36,SA,12-13-2020
478,Ampanihy,-24.7000,44.7500,74.70,55,0,7.09,MG,12-13-2020
498,Sur,22.5667,59.5289,70.45,48,0,9.51,OM,12-13-2020
552,Bara,10.3744,10.7288,70.95,24,0,4.74,NG,12-13-2020
563,Belmonte,-15.8631,-38.8828,75.78,78,0,3.58,BR,12-13-2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.

hotel_df = pd.DataFrame()
narrow_data = pd.DataFrame(idea_weather_df)
hotel_df['City'] = narrow_data['City']
hotel_df['Country'] = narrow_data['Country']
hotel_df['Lat'] = narrow_data['Latitude']
hotel_df['Lng'] = narrow_data['Longitude']
hotel_df['Hotel Name'] = "" #Store the first Hotel result into the DataFrame.
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
210,Mandera,KE,3.9366,41.8670,
251,Betioky,MG,-23.7167,44.3833,
252,Kodār,IN,16.9833,79.9667,
464,Jizan,SA,17.3333,42.6667,
478,Ampanihy,MG,-24.7000,44.7500,


In [87]:
# use iterrows to iterate through pandas dataframe
#03-05-Stu_Google_Complex
#03-04-Ins_NearestRestr

for index, row in hotel_df.iterrows():

    #get hotel type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    #assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    
    #Set parameters to search for hotels with 5000 meters. #Hit the Google Places API for each city's coordinates.
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']

    
    try:
        hotelname = response['results'][10]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 210: Mandera.
Closest hotel to Mandera at 3.9366 , 41.867 is M-Pesa Olkitira Communications Agency Ltd.
------------
Retrieving Results for Index 251: Betioky.
Closest hotel to Betioky at -23.7167 , 44.3833 is RadioFeon'nyOnilahy.
------------
Retrieving Results for Index 252: Kodār.
Closest hotel to Kodār at 16.9833 , 79.9667 is HP Petrol Bunk.
------------
Retrieving Results for Index 464: Jizan.
Missing field/result... skipping.
------------
Retrieving Results for Index 478: Ampanihy.
Closest hotel to Ampanihy at -24.7 , 44.75 is Hotel Restaurant ANGORA.
------------
Retrieving Results for Index 498: Sur.
Closest hotel to Sur at 22.5667 , 59.5289 is Pizza Hut.
------------
Retrieving Results for Index 552: Bara.
Missing field/result... skipping.
------------
Retrieving Results for Index 563: Belmonte.
Closest hotel to Belmonte at -15.8631 , -38.8828 is CEPLAC Comissão Executiva Plano Lavoura Cacaueira.
------------


In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# locations
# hotel_info


In [89]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
210,Mandera,KE,3.9366,41.8670,M-Pesa Olkitira Communications Agency Ltd
251,Betioky,MG,-23.7167,44.3833,RadioFeon'nyOnilahy
252,Kodār,IN,16.9833,79.9667,HP Petrol Bunk
464,Jizan,SA,17.3333,42.6667,
478,Ampanihy,MG,-24.7000,44.7500,Hotel Restaurant ANGORA
498,Sur,OM,22.5667,59.5289,Pizza Hut
552,Bara,NG,10.3744,10.7288,
563,Belmonte,BR,-15.8631,-38.8828,CEPLAC Comissão Executiva Plano Lavoura Cacaueira


In [90]:
# Add marker layer ontop of heat map
#03-06-Evr_Jupyter_Gmaps
 
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level= 2,center= (5,25))
fig

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# # save the plot
# plt.savefig("../output_data/map (1)")

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…